# Chapter 26: GitHub Actions

While Chapter 25 explored Jenkins as a self-hosted, extensible automation platform, this chapter examines GitHub Actions—the cloud-native, event-driven CI/CD service tightly integrated into the GitHub ecosystem. Unlike Jenkins' persistent master-agent architecture, GitHub Actions operates as a serverless compute service that responds to repository events, executing workflows in ephemeral, containerized environments.

GitHub Actions eliminates infrastructure management overhead while providing native integration with pull requests, issues, and GitHub Packages. Understanding its workflow syntax, runner ecosystem, and security model enables teams to implement sophisticated CI/CD pipelines without maintaining build servers, while self-hosted runners offer escape hatches for specialized workloads requiring custom hardware or network access.

## 26.1 GitHub Actions Concepts

GitHub Actions is built on three foundational abstractions: workflows, jobs, and steps. Understanding these concepts is essential for architecting efficient automation.

### Workflows

Workflows are automated processes defined in YAML files stored in `.github/workflows/`. Each workflow:
- Triggers based on GitHub events (push, pull request, schedule)
- Executes one or more jobs
- Runs in fresh virtual environments (runners)
- Provides execution logs and artifacts in the GitHub UI

```yaml
# .github/workflows/ci.yml
name: Continuous Integration  # Workflow name displayed in GitHub UI

on:  # Trigger configuration
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]
  schedule:
    - cron: '0 0 * * 0'  # Weekly on Sunday

jobs:  # Collection of jobs to execute
  build:
    runs-on: ubuntu-latest  # Execution environment
    steps:
      - uses: actions/checkout@v4  # Prebuilt action
      - name: Install dependencies
        run: npm ci  # Shell command
```

### Event-Driven Architecture

Unlike Jenkins' polling or webhook-triggered jobs, GitHub Actions natively responds to 30+ GitHub events:

```yaml
on:
  # Code events
  push:
    branches: ['main', 'release/*']
    paths: ['src/**', '!src/docs/**']  # Only when src changes, ignore docs
  
  pull_request:
    types: [opened, synchronize, reopened, ready_for_review]
  
  # Issue events (for automation)
  issues:
    types: [opened, labeled]
  
  # Repository events
  release:
    types: [published]
  
  # External events
  repository_dispatch:  # Webhook-triggered from external systems
    types: [deploy-staging]
  
  # Manual triggers
  workflow_dispatch:  # Manual button in GitHub UI
    inputs:
      environment:
        description: 'Deployment environment'
        required: true
        default: 'staging'
        type: choice
        options:
          - staging
          - production
      debug_enabled:
        description: 'Enable debug logging'
        required: false
        type: boolean
```

### Execution Model

GitHub Actions executes workflows with specific constraints:
- **Timeouts**: Default 360 minutes (6 hours), configurable down to 1 minute
- **Concurrency**: Free tier allows 20 concurrent jobs (Linux), 5 (macOS), 120 (Windows)
- **Billing**: Linux runners free for public repos, 2,000 minutes/month for private repos (free tier)
- **Isolation**: Each job runs in a fresh VM or container; no persistence between runs except via caching/artifacts

## 26.2 Workflow Syntax

GitHub Actions workflows use YAML with a specific schema defining the execution pipeline.

### Basic Structure

```yaml
name: Deployment Pipeline

on:
  push:
    branches: [main]

env:  # Global environment variables
  NODE_VERSION: '20'
  REGISTRY: ghcr.io

jobs:
  build:
    runs-on: ubuntu-latest  # Runner specification
    
    env:  # Job-specific variables
      BUILD_ID: ${{ github.run_id }}
    
    outputs:  # Data passed to downstream jobs
      image_tag: ${{ steps.build.outputs.tag }}
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for proper versioning
      
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: ${{ env.NODE_VERSION }}
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Build application
        run: npm run build
        continue-on-error: false  # Fail job if this step fails
      
      - name: Upload artifacts
        uses: actions/upload-artifact@v4
        with:
          name: build-output
          path: |
            dist/
            !dist/**/*.map  # Exclude source maps
          retention-days: 5
  
  test:
    needs: build  # Depends on build job
    runs-on: ubuntu-latest
    strategy:
      matrix:
        test-group: [unit, integration, e2e]
    steps:
      - uses: actions/checkout@v4
      
      - name: Download artifacts
        uses: actions/download-artifact@v4
        with:
          name: build-output
          path: dist/
      
      - name: Run tests
        run: npm run test:${{ matrix.test-group }}
```

### Contexts and Expressions

GitHub Actions provides rich contexts for dynamic workflow behavior:

```yaml
jobs:
  deploy:
    runs-on: ubuntu-latest
    # Conditional job execution
    if: github.ref == 'refs/heads/main' && github.event_name == 'push'
    
    steps:
      - name: Debug information
        run: |
          echo "Repository: ${{ github.repository }}"
          echo "Actor: ${{ github.actor }}"
          echo "SHA: ${{ github.sha }}"
          echo "Event: ${{ github.event_name }}"
          echo "Workflow: ${{ github.workflow }}"
      
      - name: Production deployment (only for specific users)
        if: contains('["admin1","admin2"]', github.actor)
        run: echo "Deploying to production..."
      
      - name: Check commit message
        if: contains(github.event.head_commit.message, '[skip ci]') == false
        run: echo "Proceeding with CI..."
      
      - name: Calculate next version
        id: semver
        run: |
          # Use GitHub Script action for complex logic
          echo "version=1.2.3" >> $GITHUB_OUTPUT
      
      - name: Use calculated version
        run: echo "Version is ${{ steps.semver.outputs.version }}"
```

### Advanced YAML Features

**Anchors and Aliases** (limited support, prefer composite actions):
```yaml
# Reusable step configuration (not natively supported, use composite actions instead)
defaults:
  run:
    shell: bash
    working-directory: ./src

# Service containers for integration testing
services:
  postgres:
    image: postgres:15-alpine
    env:
      POSTGRES_PASSWORD: postgres
    options: >-
      --health-cmd pg_isready
      --health-interval 10s
      --health-timeout 5s
      --health-retries 5
    ports:
      - 5432:5432
  
  redis:
    image: redis:7-alpine
    ports:
      - 6379:6379
```

## 26.3 Events and Triggers

The power of GitHub Actions lies in its deep integration with GitHub events, enabling sophisticated automation beyond traditional CI.

### Pull Request Automation

```yaml
name: PR Automation

on:
  pull_request:
    types: [opened, synchronize, reopened, labeled, unlabeled]

jobs:
  triage:
    runs-on: ubuntu-latest
    steps:
      - name: Add labels based on file paths
        uses: actions/labeler@v4
        with:
          configuration-path: .github/labeler.yml
      
      - name: Assign reviewers
        uses:kentaro-m/auto-assign-action@v1.2.5
        with:
          configuration-path: .github/auto_assign.yml
      
      - name: Size labeling
        uses: codelytv/pr-size-labeler@v1
        with:
          xs_max_size: '10'
          s_max_size: '100'
          m_max_size: '500'
          l_max_size: '1000'
          fail_if_xl: 'false'
          message_if_xl: >
            This PR exceeds the recommended size of 1000 lines.
            Please consider splitting it into smaller chunks.

  security-scan:
    if: contains(github.event.pull_request.labels.*.name, 'security-review')
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - name: Run security analysis
        run: |
          # Additional security checks for sensitive changes
          npm audit --audit-level=moderate
```

### Scheduled and Manual Workflows

```yaml
name: Maintenance Tasks

on:
  schedule:
    # Run at 2 AM UTC on Sundays
    - cron: '0 2 * * 0'
  
  workflow_dispatch:
    inputs:
      task:
        description: 'Maintenance task to run'
        required: true
        type: choice
        options:
          - clean-old-artifacts
          - update-dependencies
          - security-audit
      dry_run:
        description: 'Dry run mode'
        type: boolean
        default: true

jobs:
  maintenance:
    runs-on: ubuntu-latest
    steps:
      - name: Execute task
        run: |
          echo "Running ${{ github.event.inputs.task }}"
          echo "Dry run: ${{ github.event.inputs.dry_run }}"
```

### Cross-Repository Triggers

```yaml
# Trigger workflow in another repository
name: Trigger Downstream

on:
  push:
    branches: [main]

jobs:
  notify:
    runs-on: ubuntu-latest
    steps:
      - name: Trigger deployment in ops repo
        uses: peter-evans/repository-dispatch@v2
        with:
          token: ${{ secrets.PAT }}
          repository: company/infrastructure
          event-type: deploy-trigger
          client-payload: |
            {
              "service": "api",
              "sha": "${{ github.sha }}",
              "ref": "${{ github.ref }}",
              "actor": "${{ github.actor }}"
            }
```

**Receiving Repository (infrastructure repo):**
```yaml
name: Deploy Service

on:
  repository_dispatch:
    types: [deploy-trigger]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Deploy ${{ github.event.client_payload.service }}
        run: |
          echo "Deploying commit ${{ github.event.client_payload.sha }}"
          # Deployment logic here
```

## 26.4 Jobs and Steps

Jobs define the execution units, while steps define the sequential tasks within each job.

### Job Dependencies and Parallelization

```yaml
jobs:
  # Job 1: Build (runs immediately)
  build:
    runs-on: ubuntu-latest
    outputs:
      image_digest: ${{ steps.docker_build.outputs.digest }}
    steps:
      - uses: actions/checkout@v4
      - name: Build Docker image
        id: docker_build
        run: |
          docker build -t myapp:${{ github.sha }} .
          echo "digest=$(docker inspect --format='{{index .RepoDigests 0}}' myapp:${{ github.sha }})" >> $GITHUB_OUTPUT

  # Job 2 & 3: Run in parallel after build completes
  test-unit:
    needs: build  # Wait for build
    runs-on: ubuntu-latest
    steps:
      - run: echo "Running unit tests"

  test-integration:
    needs: build
    runs-on: ubuntu-latest
    services:
      database:
        image: postgres:15
    steps:
      - run: echo "Running integration tests"

  # Job 4: Runs after both test jobs complete
  deploy:
    needs: [test-unit, test-integration]  # Array dependency
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    steps:
      - run: echo "Deploying..."
        env:
          IMAGE_DIGEST: ${{ needs.build.outputs.image_digest }}
```

### Reusable Steps (Composite Actions)

For steps reused within a workflow:

```yaml
# .github/actions/setup-node/action.yml
name: 'Setup Node and Dependencies'
description: 'Sets up Node.js, caches dependencies, and installs them'

inputs:
  node-version:
    description: 'Node.js version'
    required: false
    default: '20'
  working-directory:
    description: 'Directory containing package.json'
    required: false
    default: '.'

runs:
  using: "composite"
  steps:
    - name: Setup Node.js
      uses: actions/setup-node@v4
      with:
        node-version: ${{ inputs.node-version }}
        cache: 'npm'
        cache-dependency-path: ${{ inputs.working-directory }}/package-lock.json
    
    - name: Install dependencies
      shell: bash
      working-directory: ${{ inputs.working-directory }}
      run: npm ci
      
    - name: Verify install
      shell: bash
      working-directory: ${{ inputs.working-directory }}
      run: npm list
```

**Usage in workflow:**
```yaml
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup
        uses: ./.github/actions/setup-node
        with:
          node-version: '18'
          working-directory: './frontend'
```

## 26.5 Actions and Reusable Workflows

GitHub Actions ecosystem provides 20,000+ prebuilt actions, while reusable workflows enable organization-level standardization.

### Using Marketplace Actions

```yaml
jobs:
  security:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      # Official actions
      - uses: actions/setup-node@v4
        with:
          node-version: '20'
      
      # Third-party actions (pin to specific commit for security)
      - uses: aquasecurity/trivy-action@master
        with:
          scan-type: 'fs'
          format: 'sarif'
          output: 'trivy-results.sarif'
      
      # Community actions (verify publisher)
      - name: Slack notification
        uses: 8398a7/action-slack@v3
        with:
          status: ${{ job.status }}
          channel: '#deployments'
        env:
          SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}
```

**Security Best Practice:** Pin actions to specific SHA rather than floating tags:
```yaml
# Instead of @v4
- uses: actions/checkout@b4ffde65f46336ab88eb53be808477a3936bae11  # v4.1.1

# Dependabot can update these
```

### Reusable Workflows (Organization Level)

Define standard pipelines in a central repository:

```yaml
# .github/workflows/docker-build.yml in devops/shared-workflows repo
name: Docker Build and Push

on:
  workflow_call:
    inputs:
      image_name:
        required: true
        type: string
      dockerfile:
        required: false
        type: string
        default: './Dockerfile'
      platforms:
        required: false
        type: string
        default: 'linux/amd64'
    secrets:
      registry_username:
        required: true
      registry_password:
        required: true

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Set up QEMU
        uses: docker/setup-qemu-action@v3
      
      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3
      
      - name: Login to Registry
        uses: docker/login-action@v3
        with:
          username: ${{ secrets.registry_username }}
          password: ${{ secrets.registry_password }}
      
      - name: Build and push
        uses: docker/build-push-action@v5
        with:
          context: .
          file: ${{ inputs.dockerfile }}
          platforms: ${{ inputs.platforms }}
          push: true
          tags: |
            ${{ inputs.image_name }}:${{ github.sha }}
            ${{ inputs.image_name }}:latest
          cache-from: type=gha
          cache-to: type=gha,mode=max
```

**Calling the reusable workflow:**
```yaml
# .github/workflows/ci.yml in application repo
name: CI

on:
  push:
    branches: [main]

jobs:
  build:
    uses: company/shared-workflows/.github/workflows/docker-build.yml@main
    with:
      image_name: ghcr.io/company/myapp
      platforms: 'linux/amd64,linux/arm64'
    secrets:
      registry_username: ${{ github.actor }}
      registry_password: ${{ secrets.GITHUB_TOKEN }}
```

## 26.6 Docker in GitHub Actions

Building and publishing Docker images is a primary use case, with Actions providing optimized caching and multi-platform support.

### Building Images

```yaml
name: Docker Build

on:
  push:
    branches: [main]
    tags: ['v*']

jobs:
  docker:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      
      - name: Docker meta
        id: meta
        uses: docker/metadata-action@v5
        with:
          images: |
            ghcr.io/${{ github.repository }}
          tags: |
            type=ref,event=branch
            type=semver,pattern={{version}}
            type=sha,prefix=,suffix=,format=short
      
      - name: Set up QEMU
        uses: docker/setup-qemu-action@v3
      
      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3
      
      - name: Login to GHCR
        uses: docker/login-action@v3
        with:
          registry: ghcr.io
          username: ${{ github.actor }}
          password: ${{ secrets.GITHUB_TOKEN }}
      
      - name: Build and push
        uses: docker/build-push-action@v5
        with:
          context: .
          push: ${{ github.event_name != 'pull_request' }}
          tags: ${{ steps.meta.outputs.tags }}
          labels: ${{ steps.meta.outputs.labels }}
          platforms: linux/amd64,linux/arm64
          cache-from: type=gha
          cache-to: type=gha,mode=max
          sbom: true  # Generate SBOM attestation
          provenance: true  # Generate SLSA provenance
```

### Caching Strategies

**GitHub Actions Cache (gha):**
```yaml
- name: Build with cache
  uses: docker/build-push-action@v5
  with:
    cache-from: type=gha
    cache-to: type=gha,mode=max
```

**Registry-based caching:**
```yaml
- name: Build with registry cache
  uses: docker/build-push-action@v5
  with:
    cache-from: type=registry,ref=ghcr.io/company/myapp:buildcache
    cache-to: type=registry,ref=ghcr.io/company/myapp:buildcache,mode=max
```

### Docker Compose Testing

```yaml
jobs:
  integration:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Start services
        run: docker-compose -f docker-compose.test.yml up -d
      
      - name: Wait for health
        run: |
          sleep 10
          docker-compose ps
      
      - name: Run tests
        run: docker-compose exec -T app npm test
      
      - name: Cleanup
        if: always()
        run: docker-compose down -v
```

## 26.7 Kubernetes Deployment with Actions

Deploying to Kubernetes from GitHub Actions requires authentication and deployment tooling.

### Kubectl and Helm Deployment

```yaml
name: Deploy to Kubernetes

on:
  push:
    branches: [main]

jobs:
  deploy:
    runs-on: ubuntu-latest
    permissions:
      contents: read
      id-token: write  # Required for OIDC
    
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      
      - name: Configure AWS Credentials (OIDC)
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: arn:aws:iam::123456789012:role/GitHubActionsRole
          aws-region: us-east-1
      
      - name: Update kubeconfig
        run: aws eks update-kubeconfig --name production-cluster
      
      - name: Deploy with Helm
        run: |
          helm upgrade --install myapp ./chart \
            --namespace production \
            --set image.tag=${{ github.sha }} \
            --set image.repository=${{ env.REGISTRY }}/myapp \
            --atomic \
            --timeout 5m \
            --cleanup-on-fail
      
      - name: Verify deployment
        run: |
          kubectl rollout status deployment/myapp -n production
          kubectl get pods -n production -l app=myapp
```

### GitOps with ArgoCD

Trigger ArgoCD synchronization via GitHub Actions:

```yaml
jobs:
  update-manifests:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout source
        uses: actions/checkout@v4
      
      - name: Update image tag
        run: |
          yq eval '.spec.template.spec.containers[0].image = "${{ env.REGISTRY }}/myapp:${{ github.sha }}"' -i k8s/deployment.yaml
      
      - name: Commit changes
        uses: stefanzweifel/git-auto-commit-action@v5
        with:
          commit_message: "deploy: update image to ${{ github.sha }}"
          branch: main
          file_pattern: 'k8s/*.yaml'
      
      - name: Sync ArgoCD
        uses: clowdhaus/argo-cd-action@main
        with:
          command: app sync myapp
          options: --prune --async
        env:
          ARGOCD_AUTH_TOKEN: ${{ secrets.ARGOCD_TOKEN }}
```

## 26.8 Self-Hosted Runners

While GitHub-hosted runners provide convenience, self-hosted runners offer customization, specialized hardware, and private network access.

### Runner Configuration

```yaml
# .github/workflows/self-hosted.yml
jobs:
  gpu-tests:
    runs-on: [self-hosted, gpu, linux]  # Labels matching runner
    steps:
      - uses: actions/checkout@v4
      
      - name: Run CUDA tests
        run: |
          nvidia-smi
          python train_model.py
      
      - name: Cleanup
        if: always()
        run: rm -rf /tmp/training-data/*
```

### Runner Deployment on Kubernetes

Using Actions Runner Controller (ARC):

```yaml
# Install ARC: https://github.com/actions/actions-runner-controller
apiVersion: actions.summerwind.dev/v1alpha1
kind: RunnerDeployment
metadata:
  name: production-runners
  namespace: actions-runner-system
spec:
  replicas: 3
  template:
    spec:
      repository: company/myapp
      labels:
        - self-hosted
        - production
      containers:
      - name: runner
        image: summerwind/actions-runner:latest
        resources:
          limits:
            cpu: "4000m"
            memory: "8Gi"
---
apiVersion: actions.summerwind.dev/v1alpha1
kind: HorizontalRunnerAutoscaler
metadata:
  name: runner-autoscaler
spec:
  scaleTargetRef:
    name: production-runners
  minReplicas: 1
  maxReplicas: 10
  metrics:
  - type: PercentageRunnersBusy
    scaleUpThreshold: '0.75'
    scaleDownThreshold: '0.25'
    scaleUpFactor: '2'
    scaleDownFactor: '0.5'
```

## 26.9 Secrets and Security

GitHub Actions provides robust secret management and modern security features like OIDC for cloud authentication.

### Secret Management

```yaml
jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Use secrets
        env:
          API_KEY: ${{ secrets.PRODUCTION_API_KEY }}
          DATABASE_URL: ${{ secrets.DATABASE_URL }}
        run: |
          echo "Deploying with API key..."
          # Secrets are masked in logs automatically
      
      - name: Mask additional values
        run: |
          echo "::add-mask::${{ steps.generate.outputs.token }}"
```

**Environment Protection:**
```yaml
jobs:
  deploy-production:
    runs-on: ubuntu-latest
    environment:
      name: production
      url: https://api.company.com
    steps:
      - name: Deploy
        run: echo "Deploying to production"
```

**Environment Configuration:**
- Required reviewers
- Wait timer
- Deployment branches (only main, only tags)
- Environment secrets (override repository secrets)

### OIDC Token Authentication

Avoid long-lived credentials using OpenID Connect:

```yaml
jobs:
  aws-deploy:
    permissions:
      id-token: write  # Required for OIDC
      contents: read
    steps:
      - name: Configure AWS Credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: arn:aws:iam::123456789012:role/GitHubActionsRole
          aws-region: us-east-1
      
      - name: Login to Amazon ECR
        id: login-ecr
        uses: aws-actions/amazon-ecr-login@v2
      
      - name: Build and push
        run: |
          docker build -t ${{ steps.login-ecr.outputs.registry }}/myapp:${{ github.sha }} .
          docker push ${{ steps.login-ecr.outputs.registry }}/myapp:${{ github.sha }}
```

**Trust Policy (AWS IAM):**
```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Federated": "arn:aws:iam::123456789012:oidc-provider/token.actions.githubusercontent.com"
      },
      "Action": "sts:AssumeRoleWithWebIdentity",
      "Condition": {
        "StringEquals": {
          "token.actions.githubusercontent.com:aud": "sts.amazonaws.com"
        },
        "StringLike": {
          "token.actions.githubusercontent.com:sub": "repo:company/myapp:ref:refs/heads/main"
        }
      }
    }
  ]
}
```

## 26.10 Advanced Patterns

### Matrix Builds with Inclusion/Exclusion

```yaml
strategy:
  fail-fast: false
  matrix:
    os: [ubuntu-latest, windows-latest, macos-latest]
    node: [18, 20]
    include:
      # Add specific combination
      - os: ubuntu-latest
        node: 20
        experimental: true
    exclude:
      # Remove invalid combination
      - os: macos-latest
        node: 18
```

### Conditional Matrix

```yaml
jobs:
  test:
    strategy:
      matrix:
        test-type: [unit, integration]
    steps:
      - if: matrix.test-type == 'integration'
        name: Setup database
        run: docker run -d postgres:15
      
      - name: Run ${{ matrix.test-type }} tests
        run: npm run test:${{ matrix.test-type }}
```

### Caching Dependencies

```yaml
- name: Cache Node modules
  uses: actions/cache@v3
  with:
    path: ~/.npm
    key: ${{ runner.os }}-node-${{ hashFiles('**/package-lock.json') }}
    restore-keys: |
      ${{ runner.os }}-node-
```

### Job Outputs and Dynamic Matrices

```yaml
jobs:
  prepare:
    runs-on: ubuntu-latest
    outputs:
      matrix: ${{ steps.set-matrix.outputs.matrix }}
    steps:
      - id: set-matrix
        run: |
          JSON=$(find services -name package.json -exec dirname {} \; | jq -R . | jq -s .)
          echo "matrix=$JSON" >> $GITHUB_OUTPUT
  
  build:
    needs: prepare
    runs-on: ubuntu-latest
    strategy:
      matrix:
        service: ${{ fromJson(needs.prepare.outputs.matrix) }}
    steps:
      - name: Build ${{ matrix.service }}
        run: cd ${{ matrix.service }} && docker build .
```

---

## Chapter Summary and Preview

In this chapter, we examined GitHub Actions as a cloud-native, event-driven CI/CD platform contrasting with Jenkins' self-hosted architecture. We explored the workflow syntax defining jobs, steps, and triggers, leveraging 30+ GitHub events from push operations to repository dispatches for sophisticated automation. The job execution model enables parallelization through matrices and dependency chains, while service containers facilitate integration testing with real dependencies. Reusable workflows and composite actions provide organization-level standardization without the infrastructure overhead of shared libraries. Docker integration through official actions enables multi-platform builds with GitHub Actions cache or registry-based layer caching, generating SBOMs and provenance attestations for supply chain security. Kubernetes deployment patterns demonstrated OIDC authentication eliminating long-lived credentials, alongside GitOps integration with ArgoCD. Self-hosted runners via Actions Runner Controller offer elastic compute for specialized workloads requiring GPU access or private network connectivity. Security patterns emphasized environment protection rules, required reviewers, and OpenID Connect token authentication for seamless, passwordless cloud integration.

**Key Takeaways:**
- Leverage GitHub Actions' event-driven nature to trigger workflows not just on code changes but on issues, releases, and repository dispatches, enabling comprehensive DevOps automation beyond traditional CI/CD
- Use reusable workflows centralized in dedicated repositories to enforce organizational standards (security scanning, approval gates) across hundreds of microservices without duplicating YAML
- Implement OIDC token authentication (aws-actions/configure-aws-credentials, google-github-actions/auth) to eliminate long-lived cloud credentials, using short-lived tokens scoped to specific repository branches and workflows
- Prefer GitHub-hosted runners for standard workloads to minimize operational overhead, reserving self-hosted runners for specialized requirements (GPU, private network access, specific licensing) with strict labeling and cleanup procedures
- Utilize environment protection rules with required reviewers and deployment branches for production deployments, ensuring manual gates even in automated workflows while maintaining audit trails

**Next Chapter Preview:**
Chapter 27: GitLab CI/CD explores GitLab's integrated DevOps platform, examining its YAML syntax, pipeline visualization, and native Kubernetes integration through GitLab Runners. We will analyze GitLab's unique features including the Merge Train for serializing merges, Auto DevOps for zero-configuration pipelines, and built-in security scanning (SAST/DAST/Dependency Scanning) that operates without external tool integration. This chapter covers GitLab's approach to pipeline as code, artifact management with integrated Container Registry, and deployment strategies using environments and review apps, providing comprehensive coverage of the third major CI/CD platform alongside Jenkins and GitHub Actions.